In [1]:
from utils.high_recall_information_retrieval import HRSystem
system = HRSystem(from_scratch=False, 
                  session_name='maiso',
                  finish_function=None,
                  print_function=print,
                 )  

[INIT] Starting system...
[INIT] Loading data...
[INIT] Loading/training models...
[INIT] System started.


In [ ]:
%%time
system._estimated_relevant_u()

2022-08-02 17:57:33.432171 [ INFO  ] [0,80530)
2022-08-02 17:58:04.535700 [ INFO  ] [80530,161060)
2022-08-02 17:58:55.343480 [ INFO  ] [161060,241590)
2022-08-02 17:59:43.143749 [ INFO  ] [241590,322120)
2022-08-02 18:00:32.904365 [ INFO  ] [322120,402650)
2022-08-02 18:01:23.988342 [ INFO  ] [402650,483180)
2022-08-02 18:02:18.384210 [ INFO  ] [483180,563710)
2022-08-02 18:03:10.488023 [ INFO  ] [563710,644240)
2022-08-02 18:03:58.456599 [ INFO  ] [644240,724770)
2022-08-02 18:04:50.202400 [ INFO  ] [724770,805300)
2022-08-02 18:05:43.222912 [ INFO  ] [805300,885830)
2022-08-02 18:06:41.094953 [ INFO  ] [885830,966360)
2022-08-02 18:11:14.384829 [ INFO  ] [966360,1046890)
2022-08-02 18:15:48.274300 [ INFO  ] [1046890,1127420)
2022-08-02 18:20:20.884460 [ INFO  ] [1127420,1207950)
2022-08-02 18:24:54.068832 [ INFO  ] [1207950,1288480)
2022-08-02 18:29:27.382048 [ INFO  ] [1288480,1369010)
2022-08-02 18:34:00.664631 [ INFO  ] [1369010,1449540)
2022-08-02 18:38:34.919098 [ INFO  ] [1449

In [2]:
%%time
system._estimated_relevant_u()

[0, 80530)
[80530, 161060)
[161060, 241590)
[241590, 322120)
[322120, 402650)
[402650, 483180)
[483180, 563710)
[563710, 644240)
[644240, 724770)
[724770, 805300)
[805300, 885830)
[885830, 966360)
[966360, 1046890)
[1046890, 1127420)
[1127420, 1207950)
[1207950, 1288480)
[1288480, 1369010)
[1369010, 1449540)
[1449540, 1530070)
[1530070, 1610600)
[1610600, 1691130)
[1691130, 1771660)
[1771660, 1852190)
[1852190, 1932720)
[1932720, 2013250)
[2013250, 2093780)
CPU times: user 3min 56s, sys: 1min 11s, total: 5min 8s
Wall time: 20min 44s


array([0.00973927, 0.10178416, 0.02832706, ..., 0.11625099, 0.09564741,
       0.02668877])

In [3]:
%%time
Quant_stopping(logistic_regression=system.term_highlighter, 
               labeled_data=system.labeled_data, 
               unlabeled_data=system.unlabeled_data, 
               recall_target=0.8,
               )

2022-08-02 14:48:10.778833 [ INFO  ] Predicting on labeled   data
2022-08-02 14:48:11.930792 [ INFO  ] Predicting on unlabeled data
working on batches of size: 80530
Working on batch: [0, 80530)
Working on batch: [80530, 161060)
Working on batch: [161060, 241590)
Working on batch: [241590, 322120)
Working on batch: [322120, 402650)
Working on batch: [402650, 483180)
Working on batch: [483180, 563710)
Working on batch: [563710, 644240)
Working on batch: [644240, 724770)
Working on batch: [724770, 805300)
Working on batch: [805300, 885830)
Working on batch: [885830, 966360)
Working on batch: [966360, 1046890)
Working on batch: [1046890, 1127420)
Working on batch: [1127420, 1207950)
Working on batch: [1207950, 1288480)
Working on batch: [1288480, 1369010)
Working on batch: [1369010, 1449540)
Working on batch: [1449540, 1530070)
Working on batch: [1530070, 1610600)
Working on batch: [1610600, 1691130)
Working on batch: [1691130, 1771660)
Working on batch: [1771660, 1852190)
Working on batc

NameError: name 'estimated_recall' is not defined

In [5]:
system.labeled_data[0].vector().shape

(10000,)

In [7]:
import numpy as np
np.vstack([system.labeled_data[0].vector(), system.labeled_data[0].vector()]).shape

(2, 10000)

In [9]:
import pickle

In [10]:
bow = pickle.load(open(system.labeled_data[0]._vector_filename(),'rb'))[3]

/tmp/ipykernel_23477/2190365107.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  bow = pickle.load(open(system.labeled_data[0]._vector_filename(),'rb'))[3]


In [16]:
from scipy import sparse
sparse.vstack([bow,bow])

<2x10000 sparse matrix of type '<class 'numpy.int32'>'
	with 290 stored elements in Compressed Sparse Row format>

In [1]:
from utils.io import info
def QuantCI_stopping(logistic_regression, labeled_data, unlabeled_data, recall_target):
    """
    Rr = <p0, p1, ...>
    Ur = <p0, p1, ...>
    """
    Rr = logistic_regression.predict(labeled_data)
    Ur = logistic_regression.predict(unlabeled_data)
    recallhat = estimated_recall(Rr, Ur)
    
    lower_bond = _upper_bound_variance(Rr, Ur)
    
    return recallhat+lower_bond>=recall_target

def Quant_stopping(logistic_regression, labeled_data, unlabeled_data, recall_target):
    info('Predicting on labeled   data')
    Rr = logistic_regression.predict(labeled_data)
    info('Predicting on unlabeled data')
    Ur = logistic_regression.predict(unlabeled_data)
    info('Estimating recall')
    recallhat = _estimated_recall(Rr, Ur)
    info(f'Estimated recall: {recallhat:5.4f}')
    return recallhat >= recall_target
    
def _estimated_recall(Rr, Ur):
    """
    Rr = <p0, p1, ...>
    Ur = <p0, p1, ...>
    """
    return np.sum(Rr) / (np.sum(Rr) + np.sum(Ur))

def _lower_bond_variance(Rr, Ur):
    term1 = np.var()
    term2 = 0
    return -(2 * np.sqrt(term1+term2))